In [404]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [405]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        self.valueType = type(value)
        self.unit = unit
    
    def printValues(self):
        for attr, value in self.__dict__.items():
            print("\nAttribute: " + str(attr))
            print("Value: ")
            print(value)

In [406]:
nArray = np.array([[1,2], [1,3], [2,3], [2,4], [3,4], [3,5], [4,5]])
nodes = Input("Node numbers", nArray, "None")
nodes.printValues()

# list of lengths corresponding to each element
# the length of element n is contained in lengt

lArray = np.array([6, 3*(m.sqrt(3)), 3, 6, 3*(m.sqrt(3)), 6, 3])
lengths = Input("Lengths", lArray, "[in]")
lengths.printValues()

# list of angles relative to positive x axis for each element
aArray = np.array([(7/6), 1, 0.5, (5/6), 1, (5/6), 0.5])
aArray *= m.pi
angles = Input("Element Angles (relative to the positive x axis)", aArray, "[rad]")
angles.printValues()


Attribute: name
Value: 
Node numbers

Attribute: value
Value: 
[[1 2]
 [1 3]
 [2 3]
 [2 4]
 [3 4]
 [3 5]
 [4 5]]

Attribute: valueType
Value: 
<class 'numpy.ndarray'>

Attribute: unit
Value: 
None

Attribute: name
Value: 
Lengths

Attribute: value
Value: 
[6.         5.19615242 3.         6.         5.19615242 6.
 3.        ]

Attribute: valueType
Value: 
<class 'numpy.ndarray'>

Attribute: unit
Value: 
[in]

Attribute: name
Value: 
Element Angles (relative to the positive x axis)

Attribute: value
Value: 
[3.66519143 3.14159265 1.57079633 2.61799388 3.14159265 2.61799388
 1.57079633]

Attribute: valueType
Value: 
<class 'numpy.ndarray'>

Attribute: unit
Value: 
[rad]


Constants Inputs
<br>Units are : kip, in, in2, ksi, kip/in

In [407]:
E = 30*(10**3)
A = 1
P1y = -2.5


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [408]:
numElems = len(nodes.value)
print(numElems)
numNodes = nodes.value.max()
print(numNodes)

7
5


displacements in mm, two reactions per node
<br>uvList -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.
<br><br>this setup is advantageous over a numnodes x 2 array as it allows for easier value access later, when many sub-arrays will be required.


In [409]:
uvList = [None]*(numNodes*2)
uv_BCs = np.array([[4,0,0],[5,0,0]])
print("displacement bcs: [node number, x disp (u), y disp (v)]")
print(uv_BCs)

displacement bcs: [node number, x disp (u), y disp (v)]
[[4 0 0]
 [5 0 0]]


In [410]:
adjust_array(uvList, uv_BCs, "displacements")

print("u & v list:")
print(uvList)

adjusting array to account for: displacements boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
Case 2 for BC 2
done adjusting displacements array!
u & v list:
[None, None, None, None, None, None, 0, 0, 0, 0]


Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [411]:
#forceList = [0]*(numNodes*2)
forceList = np.zeros(((numNodes*2), 1))

nodesWithLoads = []

force_BCs = np.array([[1, 0, P1y],[4, None, None], [5, None, None]])
print(len(force_BCs))
loadedNodes = 0
for i in range(len(force_BCs)):
    if force_BCs[i,1] or force_BCs[i,2] != None:
        loadedNodes += 1
        nodesWithLoads.append((force_BCs[i,0] - 1))


print("number of nodes with concentrated loads: {0}".format(loadedNodes))
print("at nodes: {0}".format(nodesWithLoads))

#adjust force array given concentrated loading conditions
adjust_array(forceList, force_BCs, "forces")

print("force list: ")
print(forceList)

3
number of nodes with concentrated loads: 1
at nodes: [0]
adjusting array to account for: forces boundary conditions
case 3 for BC 1
Case 2 for BC 1
case 1 for BC 2
Case 2 for BC 2
case 1 for BC 3
Case 2 for BC 3
done adjusting forces array!
force list: 
[[ 0. ]
 [-2.5]
 [ 0. ]
 [ 0. ]
 [ 0. ]
 [ 0. ]
 [ nan]
 [ nan]
 [ nan]
 [ nan]]


Global [k] matrix is a symmetric array of shape (numNodes x 2, numNodes x 2)
<br>Bar Constants are one per element, stored in a numElemes * 1 array.

In [412]:
globalK = np.zeros(((numNodes*2),(numNodes*2)))
print(globalK.shape)

barConstant = np.zeros((numElems,1))
for i in range(numElems):
    barConstant[i] = (E*A)/lengths.value[i]

print("Bar Constants = ")
print(barConstant)

(10, 10)
Bar Constants = 
[[ 5000.       ]
 [ 5773.5026919]
 [10000.       ]
 [ 5000.       ]
 [ 5773.5026919]
 [ 5000.       ]
 [10000.       ]]


## Creating the finite element equation for each Truss Element

In [413]:
for i in range(numElems):
    kMatrix = create_truss_k(angles.value[i])
    kMatrix *= barConstant[i]
    #print("k matrix:")
    #print(kMatrix)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print("Element " + str(i+1) + " starting at node: " + str(startNode) + " and ending at node: " + str(endNode))
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, globalK, 0, 0, kMatrix)
    block_array_adjust(En, Sn, globalK, 2, 0, kMatrix)
    block_array_adjust(Sn, En, globalK, 0, 2, kMatrix)
    block_array_adjust(En, En, globalK, 2, 2, kMatrix)

print(globalK)
check_Symmetric(globalK)

Element 1 starting at node: 1 and ending at node: 2
Element 2 starting at node: 1 and ending at node: 3
Element 3 starting at node: 2 and ending at node: 3
Element 4 starting at node: 2 and ending at node: 4
Element 5 starting at node: 3 and ending at node: 4
Element 6 starting at node: 3 and ending at node: 5
Element 7 starting at node: 4 and ending at node: 5
[[  9523.5026919    2165.06350946  -3750.          -2165.06350946
   -5773.5026919       0.              0.              0.
       0.              0.        ]
 [  2165.06350946   1250.          -2165.06350946  -1250.
       0.             -0.              0.              0.
       0.              0.        ]
 [ -3750.          -2165.06350946   7500.              0.
      -0.             -0.          -3750.           2165.06350946
       0.              0.        ]
 [ -2165.06350946  -1250.              0.          12500.
      -0.         -10000.           2165.06350946  -1250.
       0.              0.        ]
 [ -5773.5026919

nums = [0, 0.56, 1.3, 3, None]
j = 0
while j < numNodes*2:
    print(str(j))
    j += 1

In [417]:
print(inv(globalK))
u = np.matmul(inv(globalK), forceList)
#print(u)

[[ 0.00034641  0.          0.00017321  0.0003      0.00017321  0.0003
   1.         -0.         -0.          0.        ]
 [ 0.          0.00623923  0.00180111  0.00231962  0.0003      0.00211962
   3.46410162 -1.         -3.46410162  2.        ]
 [ 0.00017321  0.00180111  0.00073987  0.00081962  0.00017321  0.00076188
   2.         -0.57735027 -1.          0.57735027]
 [ 0.0003      0.00231962  0.00081962  0.00141962  0.0003      0.00131962
   1.73205081 -0.         -1.73205081  1.        ]
 [ 0.00017321  0.0003      0.00017321  0.0003      0.00017321  0.0003
   1.         -0.         -0.          0.        ]
 [ 0.0003      0.00211962  0.00076188  0.00131962  0.0003      0.00131962
   1.73205081 -0.         -1.73205081  1.        ]
 [ 0.          0.          0.          0.          0.          0.
   1.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          1.          0.          0.        ]
 [ 0.          0.       

This code is "method 3," begins by checking if a displacement is 0. if it is, it sets the 
corresponding force value to 0. Then, it checks for positivity/negativity within the row
that the 0 displacement corresponds to. Positive values get set to 1, while negatives
get set to 0. It also catches None (unknown) values by just incrementing the loop value.
For any other known u values, it sets the force equal to that value and proceeds to alter
the stiffness matrix in the same fashion.

In [415]:
j = 0
while j < numNodes*2:
    print(str(j))
    if j in nodesWithLoads:
        j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #globalK to reflect the changes.
    elif uvList[j] == 0:
        print("u = 0 at row: " + str(j))
        forceList[j] = 0 
        globalK[j] = 0
        globalK[j,j] = 1
        j+=1
    elif uvList[j] == None:
        print("none detected at row " + str(j))
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print("row " + str(j) + " altered")
        print(uvList[j])
        forceList[j] = uvList[j]
        for c in range(numNodes):
            if (globalK[j,c] > 0) == True:
                globalK[j,c] = 1
            else:
                globalK[j,c] = 0
        j+=1

                
print(globalK)
print(forceList)

0
2
none detected at row 2
3
none detected at row 3
4
none detected at row 4
5
none detected at row 5
6
u = 0 at row: 6
7
u = 0 at row: 7
8
u = 0 at row: 8
9
u = 0 at row: 9
[[  9523.5026919    2165.06350946  -3750.          -2165.06350946
   -5773.5026919       0.              0.              0.
       0.              0.        ]
 [  2165.06350946   1250.          -2165.06350946  -1250.
       0.             -0.              0.              0.
       0.              0.        ]
 [ -3750.          -2165.06350946   7500.              0.
      -0.             -0.          -3750.           2165.06350946
       0.              0.        ]
 [ -2165.06350946  -1250.              0.          12500.
      -0.         -10000.           2165.06350946  -1250.
       0.              0.        ]
 [ -5773.5026919       0.             -0.             -0.
   15297.00538379  -2165.06350946  -5773.5026919       0.
   -3750.           2165.06350946]
 [     0.             -0.             -0.         -1000

number of elements is the number of rows in nodes
number of nodes is the maximum number found in nodes

In [416]:
#globalK.round(0)
globalK_inv = inv(globalK)
#print(globalK_inv)

uFinal = np.matmul(globalK_inv, forceList)
print(uFinal)

[[-0.        ]
 [-0.01559808]
 [-0.00450278]
 [-0.00579904]
 [-0.00075   ]
 [-0.00529904]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
